# 少量示例的提示词模板的使用

FewShotPromptTemplate：与PromptTemplate一起使用

FewShotChatMessagePromptTemplate：与ChatPromptTemplate一起使用

Example selectors（示例选择器）

## 1、使用 FewShotPromptTemplate


举例1：未提供示例的情况

In [1]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.4)

res = chat_model.invoke("2 🦜 9是多少?")
print(res.content)

2 🦜 9 的结果取决于你所使用的运算符。如果你是想问 2 加 9，那就是 11。如果有其他特定的运算方式，请告诉我！


举例2：使用FewShotPromptTemplate

In [2]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# 创建一个PromptTemplate的实例
example_prompt = PromptTemplate.from_template(
    template="Input: {input}\nOutput: {output}"
)

# 提供一些示例
examples = [
    {"input": "北京天气怎么样", "output": "北京市"},
    {"input": "南京下雨吗", "output": "南京市"},
    {"input": "武汉热吗", "output": "武汉市"}
]

# 创建 FewShotPromptTemplate 的实例
few_shot_template = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Input: {input}\nOutput: ",  # 要放在示例后面的提示模板字符串
    input_variables=["input"]  # 传入的变量
)

# 调用
prompt = few_shot_template.invoke({"input": "天津会下雨嘛？"})

print("=== Prompt ===")
print(prompt)

=== Prompt ===
text='Input: 北京天气怎么样\nOutput: 北京市\n\nInput: 南京下雨吗\nOutput: 南京市\n\nInput: 武汉热吗\nOutput: 武汉市\n\nInput: 天津会下雨嘛？\nOutput: '


调用大模型以后：

In [3]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.4)

response = chat_model.invoke(prompt)
print(response.content)

天津市


举例3：

In [4]:
#1、创建提示模板
from langchain.prompts import PromptTemplate

# 创建提示模板，配置一个提示模板，将一个示例格式化为字符串
prompt_template = "你是一个数学专家,算式： {input} 值： {output} 使用： {description} "

# 这是一个提示模板，用于设置每个示例的格式
prompt_sample = PromptTemplate.from_template(prompt_template)

#2、提供示例
examples = [
    {"input": "2+2", "output": "4", "description": "加法运算"},
    {"input": "5-2", "output": "3", "description": "减法运算"},
]

#3、创建一个FewShotPromptTemplate对象
from langchain.prompts.few_shot import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt_sample,
    suffix="你是一个数学专家,算式: {input} 值: {output}",
    input_variables=["input", "output"]
)

print(prompt.invoke({"input": "2*5", "output": "10"}))

#4、初始化大模型，然后调用
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="gpt-4o-mini")

result = chat_model.invoke(prompt.invoke({"input": "2*5", "output": "10"}))
print(result.content)  # 使用: 乘法运算


text='你是一个数学专家,算式： 2+2 值： 4 使用： 加法运算 \n\n你是一个数学专家,算式： 5-2 值： 3 使用： 减法运算 \n\n你是一个数学专家,算式: 2*5 值: 10'
你是一个数学专家，算式： 2*5 值： 10 使用： 乘法运算 

如果需要更多算式或其他运算的示例，请告诉我！


## 2、FewShotChatMessagePromptTemplate的使用

举例1：实例化

In [5]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

# 1.示例消息格式
examples = [
    {"input": "1+1等于几？", "output": "1+1等于2"},
    {"input": "法国的首都是？", "output": "巴黎"}
]

# 2.定义示例的消息格式提示词模版
msg_example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

# 3.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=msg_example_prompt,
    examples=examples
)

# 4.输出格式化后的消息
print(few_shot_prompt.format())

Human: 1+1等于几？
AI: 1+1等于2
Human: 法国的首都是？
AI: 巴黎


举例2：

In [6]:
# 1.导入相关包
from langchain_core.prompts import (FewShotChatMessagePromptTemplate,
                                    ChatPromptTemplate)

# 2.定义示例组
examples = [
    {"input": "2🦜2", "output": "4"},
    {"input": "2🦜3", "output": "8"},
]

# 3.定义示例的消息格式提示词模版
example_prompt = ChatPromptTemplate.from_messages([
    ('human', '{input} 是多少?'),
    ('ai', '{output}')
])

# 4.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,  # 示例组
    example_prompt=example_prompt,  # 示例提示词词模版
)

# 5.输出完整提示词的消息模版
final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '你是一个数学奇才'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)

#6.提供大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.4)

chat_model.invoke(final_prompt.invoke(input="2🦜4")).content


'16'